<a href="https://colab.research.google.com/github/Arminsbss/Reinforcement-learning-N-N-grid/blob/main/reinforcement_learning_n_n_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

Grid = pd.read_excel('Grid.xlsx',header=None)
Grid = Grid.values

**States**

In [ ]:
Start_State_indexes=[29,0]
End_State_indexes=[0,29]
Grid[Start_State_indexes[0]][Start_State_indexes[1]]

-10

**Actions**

In [ ]:
# The function below returns True if the indices are within the range of the array, and False otherwise.
def is_valid_cell(get_cell,row,column):
    num_rows = len(get_cell)
    num_columns = len(get_cell[0]) if num_rows > 0 else 0
    return 0 <= row < num_rows and 0 <= column < num_columns

def Right_Action(get_cell,row,column):
  if is_valid_cell(get_cell,row,column):
    return get_cell[row,column+1]
  else:
    return "NA"

def Up_Action(get_cell,row,column):
  if is_valid_cell(get_cell,row,column):
    return get_cell[row-1,column]
  else:
    return "NA"

In [ ]:
State=Grid[Start_State_indexes[0]][Start_State_indexes[1]]
State=Up_Action(Grid,Start_State_indexes[0],Start_State_indexes[1])
print(State)

5


In [ ]:
if is_valid_cell(Grid,Start_State_indexes[0],Start_State_indexes[1]):
  print("T")
else:
  print("F")

T


**Q-Values**

Q(s, a) = (1 − α) Q(s, a) + α(R(s) + γ (max[Q(s′, a′)]))